# Batch Evaluation
## Helper functions

In [35]:
import numpy as np
from scipy.spatial.transform import Rotation as R

from evo.core.trajectory import PoseTrajectory3D
from evo.tools import file_interface
import os

from evo.core import sync
from evo.core import metrics

import pandas as pd

def read_ref_to_tum(path_ref, path_stamps):
    stamps = np.loadtxt(path_stamps, dtype = np.float64)

    with open(path_ref, "r") as f:
        lines = f.readlines()

    xyz = np.zeros((len(lines), 3), dtype = np.float64)
    quat = np.zeros((len(lines), 4), dtype = np.float64)

    for i , line in enumerate(lines):
        Tcw = np.array(list(map(float, line.split()))).reshape(4, 4)
        # Tcw[:3, 1] *= -1
        # Tcw[:3, 2] *= -1
        
        xyz[i, :] = Tcw[0:3, 3]
        
        rot = R.from_matrix(Tcw[0:3, 0:3])
        quat[i, :] = rot.as_quat()
    
    return PoseTrajectory3D(xyz, quat, stamps)

# load ref
def load_traj_ref_dict(dir, path_stamps): 
    traj_ref_dict = {}
    for filename in os.listdir(dir): 
        traj_ref_dict[filename[:-4]] = read_ref_to_tum(os.path.join(dir, filename), path_stamps)
    return traj_ref_dict

# compute stats of one given experiment
def compute_single_stat(seq, method, k, key, dir_est, traj_ref_dict, max_diff):
    line = [seq, method, k, 0, 1, 1, -1, -1]

    abs_path = os.path.join(dir_est, key + '.txt')
    if not os.path.isfile(abs_path): 
        return line 
    if not os.path.getsize(abs_path):
        return line

    traj_est = file_interface.read_tum_trajectory_file(abs_path)
    traj_ref = traj_ref_dict[seq]
    if traj_est.num_poses < 3:
        return line
    path_length_ref = traj_ref.path_length
    
    # synchronize, align and scale
    traj_ref, traj_est_aligned = sync.associate_trajectories(traj_ref, traj_est, max_diff)
    traj_est_aligned.align(traj_ref, correct_scale=True, correct_only_scale=False)

    # create metric relation and data
    pose_relation = metrics.PoseRelation.translation_part
    data = (traj_ref, traj_est_aligned)

    # process APE
    ape_metric = metrics.APE(pose_relation)
    ape_metric.process_data(data)

    ape_stat = ape_metric.get_statistic(metrics.StatisticsType.rmse)
    line[3] = traj_est_aligned.path_length / path_length_ref
    line[4] = ape_stat

    # RPE
    # synchronize, align and scale
    traj_ref, traj_est_rpe = sync.associate_trajectories(traj_ref, traj_est, max_diff)

    pose_relation = metrics.PoseRelation.rotation_angle_rad

    # normal mode
    delta = 1

    # all pairs mode
    all_pairs = False  # activate
    data = (traj_ref, traj_est_rpe)

    rpe_metric = metrics.RPE(pose_relation=pose_relation, delta=delta, all_pairs=all_pairs)
    rpe_metric.process_data(data)

    rpe_stat = rpe_metric.get_statistic(metrics.StatisticsType.rmse)
    line[5] = rpe_stat


    abs_path_feats = os.path.join(dir_est, 'keypoints', key + '_feats.txt')
    if not os.path.isfile(abs_path_feats): 
        return line 
    if not os.path.getsize(abs_path_feats):
        return line
    
    with open(abs_path_feats, 'r') as f: 
        ll = [int(l) for l in f]
        if len(ll) == 2:
            line[6:] = ll
    
    return line

In [36]:
# Reference (grund truth) preparation

dir_ref = "/media/huanglab/HuangLab10TB/slam_team/dataset/Replica/ref/"
path_stamps = "/media/huanglab/HuangLab10TB/slam_team/dataset/Replica/timestamp_20hz.txt"
traj_ref_dict = load_traj_ref_dict(dir_ref, path_stamps)

## Image perturbation (static)

In [37]:
def compute_stats(dir_est, traj_ref_dict, max_diff = 0.01): 
    # Save results, with list of ALL settings regardless of having results
    data_table = []

    methods = ["motion_blur", "defocus_blur", "gaussian_blur", "glass_blur",
            "gaussian_noise", "shot_noise", "impulse_noise", "speckle_noise", 
            "fog", "frost", "snow", "spatter",
            "brightness", "contrast", "jpeg_compression", "pixelate"]

    for method in methods: 
        for k in range(1, 6, 2): 
            # office 
            for n in range(5):
                seq = "office" + str(n)
                key = '_'.join([seq, method, str(k)])
                line = compute_single_stat(seq, method, k, key, dir_est, traj_ref_dict, max_diff)
                data_table.append(line)
            
            # room 
            for n in range(3):
                seq = "room" + str(n)
                key = '_'.join([seq, method, str(k)])
                line = compute_single_stat(seq, method, k, key, dir_est, traj_ref_dict, max_diff)
                data_table.append(line)
    # None
    # office
    for n in range(5):
        seq = "office" + str(n)
        key = '_'.join([seq, 'none', '0'])
        line = compute_single_stat(seq, 'none', '0', key, dir_est, traj_ref_dict, max_diff)
        data_table.append(line)

    # room 
    for n in range(3):
        seq = "room" + str(n)
        key = '_'.join([seq, 'none', '0'])
        line = compute_single_stat(seq, 'none', '0', key, dir_est, traj_ref_dict, max_diff)
        data_table.append(line)
    
    return data_table

In [38]:
dir_est = "/media/huanglab/HuangLab10TB/slam_team/sibo/results_replica/rgbd/rgbd1_image_perturbation/"
data_table = compute_stats(dir_est, traj_ref_dict)

In [39]:

# Create dataframe 
df = pd.DataFrame(data_table)
df.columns = ['Sequence', 'Method', 'Level', 'Length Ratio', 'APE', 'RPE', 'Ave KP', 'Ave MP']
with pd.ExcelWriter('orb_result_all.xlsx', mode='a', if_sheet_exists='replace') as writer:
    df.to_excel(writer, sheet_name='image_perturbation_static_rgbd')

# Filter by ratio of tracked lengths
# df1 = df[df['Length Ratio']>0.8].groupby('Method')['APE'].mean()
# df1 = df.groupby('Method')['Length Ratio'].mean()
# df1 = df.groupby(['Method']).agg({'Length Ratio':['mean', 'std']})
# with pd.ExcelWriter('orb_length_ratio.xlsx', mode='a', if_sheet_exists='replace') as writer1:
#     df1.to_excel(writer1, sheet_name='image_perturbation_static_rgbd')
# print(df1)
df1 = df.groupby(['Method']).agg({'APE':['mean', 'std']})
with pd.ExcelWriter('orb_ape.xlsx', mode='a', if_sheet_exists='replace') as writer1:
    df1.to_excel(writer1, sheet_name='image_perturbation_static_rgbd')
print(df1)

                       APE          
                      mean       std
Method                              
brightness        0.064932  0.141538
contrast          0.527448  0.492261
defocus_blur      0.339945  0.407585
fog               0.397061  0.480354
frost             0.640226  0.474439
gaussian_blur     0.292277  0.424510
gaussian_noise    0.469957  0.498277
glass_blur        0.211459  0.334700
impulse_noise     0.590925  0.494465
jpeg_compression  0.131869  0.191049
motion_blur       0.299639  0.364775
none              0.082256  0.179367
pixelate          0.703352  0.354472
shot_noise        0.432955  0.490291
snow              0.794685  0.408848
spatter           0.507812  0.502976
speckle_noise     0.351097  0.469371


## Image perturbation (dynamic)

In [40]:
def compute_stats(dir_est, traj_ref_dict, max_diff = 0.01): 
    # Save results, with list of ALL settings regardless of having results
    data_table = []

    methods = ["motion_blur", "defocus_blur", "gaussian_blur", "glass_blur",
            "gaussian_noise", "shot_noise", "impulse_noise", "speckle_noise", 
            "fog", "frost", "snow", "spatter",
            "brightness", "contrast", "jpeg_compression", "pixelate"]

    for method in methods: 
        # for k in range(1, 6, 2): 
            # office 
            for n in range(5):
                seq = "office" + str(n)
                key = '_'.join([seq, method, 'rand'])
                line = compute_single_stat(seq, method, 'rand', key, dir_est, traj_ref_dict, max_diff)
                data_table.append(line)
            
            # room 
            for n in range(3):
                seq = "room" + str(n)
                key = '_'.join([seq, method,'rand'])
                line = compute_single_stat(seq, method, 'rand', key, dir_est, traj_ref_dict, max_diff)
                data_table.append(line)
    
    return data_table


dir_est = "/media/huanglab/HuangLab10TB/slam_team/sibo/results_replica/rgbd/rgbd1_dyn_image_perturbation/"
data_table = compute_stats(dir_est, traj_ref_dict)

# Create dataframe 
df = pd.DataFrame(data_table)
df.columns = ['Sequence', 'Method', 'Level', 'Length Ratio', 'APE', 'RPE', 'Ave KP', 'Ave MP']
with pd.ExcelWriter('orb_result_all.xlsx', mode='a', if_sheet_exists='replace') as writer:
    df.to_excel(writer, sheet_name='image_perturbation_dyn_rgbd')

# Filter by ratio of tracked lengths
# df1 = df[df['Length Ratio']>0.8].groupby('Method')['APE'].mean()
# df1 = df.groupby('Method')['Length Ratio'].mean()
# df1 = df.groupby(['Method']).agg({'Length Ratio':['mean', 'std']})
# with pd.ExcelWriter('orb_length_ratio.xlsx', mode='a', if_sheet_exists='replace') as writer1:
#     df1.to_excel(writer1, sheet_name='image_perturbation_dyn_rgbd')
# print(df1)
df1 = df.groupby(['Method']).agg({'APE':['mean', 'std']})
with pd.ExcelWriter('orb_ape.xlsx', mode='a', if_sheet_exists='replace') as writer1:
    df1.to_excel(writer1, sheet_name='image_perturbation_dyn_rgbd')
print(df1)

                       APE          
                      mean       std
Method                              
brightness        0.065976  0.144607
contrast          0.750639  0.461729
defocus_blur      0.302988  0.434551
fog               0.659048  0.475836
frost             0.255967  0.459245
gaussian_blur     0.051508  0.079546
gaussian_noise    0.508131  0.525890
glass_blur        0.167984  0.245440
impulse_noise     0.755152  0.453372
jpeg_compression  0.645180  0.491277
motion_blur       0.279303  0.330419
pixelate          0.755638  0.294402
shot_noise        0.513253  0.520456
snow              0.875567  0.351951
spatter           0.752840  0.457676
speckle_noise     0.262411  0.455280


## Faster motion

In [49]:
def compute_stats(dir_est, traj_ref_dict, max_diff = 0.01): 
    # Save results, with list of ALL settings regardless of having results
    data_table = []

    methods = ["faster_motion"]

    for method in methods: 
        for k in [2, 4, 8]: 
            # office 
            for n in range(5):
                seq = "office" + str(n)
                key = '_'.join([seq, 'r' + str(k)])
                line = compute_single_stat(seq, method, 'r' + str(k), key, dir_est, traj_ref_dict, max_diff)
                data_table.append(line)
            
            # room 
            for n in range(3):
                seq = "room" + str(n)
                key = '_'.join([seq, 'r' + str(k)])
                line = compute_single_stat(seq, method, 'r' + str(k), key, dir_est, traj_ref_dict, max_diff)
                data_table.append(line)
    
    return data_table


dir_est = "/media/huanglab/HuangLab10TB/slam_team/sibo/results_replica/rgbd/rgbd1_faster_motion/"
data_table = compute_stats(dir_est, traj_ref_dict)

# Create dataframe 
df = pd.DataFrame(data_table)
df.columns = ['Sequence', 'Method', 'Level', 'Length Ratio', 'APE', 'RPE', 'Ave KP', 'Ave MP']
with pd.ExcelWriter('orb_result_all.xlsx', mode='a', if_sheet_exists='replace') as writer:
    df.to_excel(writer, sheet_name='faster_motion_rgbd')

# Filter by ratio of tracked lengths
# df1 = df[df['Length Ratio']>0.8].groupby('Method')['APE'].mean()
# df1 = df.groupby('Level')['Length Ratio'].mean()
df1 = df.groupby(['Level']).agg({'Length Ratio':['mean', 'std']})
with pd.ExcelWriter('orb_length_ratio.xlsx', mode='a', if_sheet_exists='replace') as writer1:
    df1.to_excel(writer1, sheet_name='faster_motion_rgbd')
print(df1)

      Length Ratio          
              mean       std
Level                       
r2        0.893777  0.080546
r4        0.909299  0.048945
r8        0.837443  0.114871


## Depth perturbation

In [41]:
def compute_stats(dir_est, traj_ref_dict, max_diff = 0.01): 
    # Save results, with list of ALL settings regardless of having results
    data_table = []

    methods = ["depth_add_gaussian_noise", "depth_add_edge_erosion", "depth_add_random_mask", "depth_range"]

    for method in methods: 
        for k in [3]: 
            # office 
            for n in range(5):
                seq = "office" + str(n)
                key = '_'.join([seq, method, str(k)])
                line = compute_single_stat(seq, method, k, key, dir_est, traj_ref_dict, max_diff)
                data_table.append(line)
            
            # room 
            for n in range(3):
                seq = "room" + str(n)
                key = '_'.join([seq, method, str(k)])
                line = compute_single_stat(seq, method, k, key, dir_est, traj_ref_dict, max_diff)
                data_table.append(line)
    
    return data_table


dir_est = "/media/huanglab/HuangLab10TB/slam_team/sibo/results_replica/rgbd/rgbd1_depth_perturbation/"
data_table = compute_stats(dir_est, traj_ref_dict)

# Create dataframe 
df = pd.DataFrame(data_table)
df.columns = ['Sequence', 'Method', 'Level', 'Length Ratio', 'APE', 'RPE', 'Ave KP', 'Ave MP']
with pd.ExcelWriter('orb_result_all.xlsx', mode='a', if_sheet_exists='replace') as writer:
    df.to_excel(writer, sheet_name='depth_perturbation')

# Filter by ratio of tracked lengths
# df1 = df[df['Length Ratio']>0.8].groupby('Method')['APE'].mean()
# df1 = df.groupby('Method')['Length Ratio'].mean()
# df1 = df.groupby(['Method']).agg({'Length Ratio':['mean', 'std']})
# with pd.ExcelWriter('orb_length_ratio.xlsx', mode='a', if_sheet_exists='replace') as writer1:
#     df1.to_excel(writer1, sheet_name='depth_perturbation')
# print(df1)
df1 = df.groupby(['Method']).agg({'APE':['mean', 'std']})
with pd.ExcelWriter('orb_ape.xlsx', mode='a', if_sheet_exists='replace') as writer1:
    df1.to_excel(writer1, sheet_name='depth_perturbation')
print(df1)

                               APE          
                              mean       std
Method                                      
depth_add_edge_erosion    0.807425  0.364686
depth_add_gaussian_noise  0.802572  0.300749
depth_add_random_mask     0.755583  0.396600
depth_range               0.994002  0.282975


## Sensor misalignment

In [51]:
def compute_stats(dir_est, traj_ref_dict, max_diff = 0.01): 
    # Save results, with list of ALL settings regardless of having results
    data_table = []

    methods = ["misalign"]

    for method in methods: 
        for k in [5, 10, 20]: 
            # office 
            for n in range(5):
                seq = "office" + str(n)
                key = '_'.join([seq, 'k' + str(k)])
                line = compute_single_stat(seq, method, 'k' + str(k), key, dir_est, traj_ref_dict, max_diff)
                data_table.append(line)
            
            # room 
            for n in range(3):
                seq = "room" + str(n)
                key = '_'.join([seq, 'k' + str(k)])
                line = compute_single_stat(seq, method, 'k' + str(k), key, dir_est, traj_ref_dict, max_diff)
                data_table.append(line)
    
    return data_table


dir_est = "/media/huanglab/HuangLab10TB/slam_team/sibo/results_replica/sensor_misalign/dynamic"
data_table = compute_stats(dir_est, traj_ref_dict)

# Create dataframe 
df = pd.DataFrame(data_table)
df.columns = ['Sequence', 'Method', 'Level', 'Length Ratio', 'APE', 'RPE', 'Ave KP', 'Ave MP']
with pd.ExcelWriter('orb_result_all.xlsx', mode='a', if_sheet_exists='replace') as writer:
    df.to_excel(writer, sheet_name='sensor_misalign_dynamic')

# Filter by ratio of tracked lengths
# df1 = df[df['Length Ratio']>0.8].groupby('Method')['APE'].mean()
# df1 = df.groupby('Level')['Length Ratio'].mean()
df1 = df.groupby(['Level']).agg({'Length Ratio':['mean', 'std']})
with pd.ExcelWriter('orb_length_ratio.xlsx', mode='a', if_sheet_exists='replace') as writer1:
    df1.to_excel(writer1, sheet_name='sensor_misalign_dynamic')
print(df1)

      Length Ratio          
              mean       std
Level                       
k10       0.942307  0.050416
k20       0.948122  0.041312
k5        0.955365  0.039133


## Trajectory perturbation

In [52]:

# compute stats of one given experiment
def compute_single_stat(seq, method, k, key, dir_est, dir_ref, max_diff):
    line = [seq, method, k, 0, 1, 1, -1, -1]

    abs_path = os.path.join(dir_est, key + '.txt')
    if not os.path.isfile(abs_path): 
        return line 
    if not os.path.getsize(abs_path):
        return line

    traj_est = file_interface.read_tum_trajectory_file(abs_path)
    abs_path_ref = os.path.join(dir_ref, "Replica_" + key + "_normal", "traj.txt")
    traj_ref = read_ref_to_tum(abs_path_ref, path_stamps)

    if traj_est.num_poses < 3:
        return line
    path_length_ref = traj_ref.path_length
    
    # synchronize, align and scale
    traj_ref, traj_est_aligned = sync.associate_trajectories(traj_ref, traj_est, max_diff)
    traj_est_aligned.align(traj_ref, correct_scale=True, correct_only_scale=False)

    # create metric relation and data
    pose_relation = metrics.PoseRelation.translation_part
    data = (traj_ref, traj_est_aligned)

    # process APE
    ape_metric = metrics.APE(pose_relation)
    ape_metric.process_data(data)

    ape_stat = ape_metric.get_statistic(metrics.StatisticsType.rmse)
    line[3] = traj_est_aligned.path_length / path_length_ref
    line[4] = ape_stat

    # RPE
    # synchronize, align and scale
    traj_ref, traj_est_rpe = sync.associate_trajectories(traj_ref, traj_est, max_diff)

    pose_relation = metrics.PoseRelation.rotation_angle_rad

    # normal mode
    delta = 1

    # all pairs mode
    all_pairs = False  # activate
    data = (traj_ref, traj_est_rpe)

    rpe_metric = metrics.RPE(pose_relation=pose_relation, delta=delta, all_pairs=all_pairs)
    rpe_metric.process_data(data)

    rpe_stat = rpe_metric.get_statistic(metrics.StatisticsType.rmse)
    line[5] = rpe_stat


    abs_path_feats = os.path.join(dir_est, 'keypoints', key + '_feats.txt')
    if not os.path.isfile(abs_path_feats): 
        return line 
    if not os.path.getsize(abs_path_feats):
        return line
    
    with open(abs_path_feats, 'r') as f: 
        ll = [int(l) for l in f]
        if len(ll) == 2:
            line[6:] = ll
    
    return line


def compute_stats(dir_est, dir_ref, max_diff = 0.01): 
    # Save results, with list of ALL settings regardless of having results
    data_table = []

    for ri in [0, 1, 3, 5]: 
        for ti in ["0", "0dot0125", "0dot025", "0dot05"]: 
            # office 
            for n in range(5):
                seq = "office" + str(n)
                key = '_'.join(["office", str(n), 'rot', str(ri), 'tran', ti])
                line = compute_single_stat(seq, ri, ti, key, dir_est, dir_ref, max_diff)
                data_table.append(line)
            
            # room 
            for n in range(3):
                seq = "room" + str(n)
                key = '_'.join(["room", str(n), 'rot', str(ri), 'tran', ti])
                line = compute_single_stat(seq, ri, ti, key, dir_est, dir_ref, max_diff)
                data_table.append(line)
    
    return data_table


dir_ref = "/media/huanglab/HuangLab10TB/slam_team/dataset/Replica-Dataset-Traj-Perturbed_simple/"
dir_est = "/media/huanglab/HuangLab10TB/slam_team/sibo/results_replica/perturb_trajectory/rgbd1/"
data_table = compute_stats(dir_est, dir_ref)

# Create dataframe 
df = pd.DataFrame(data_table)
df.columns = ['Sequence', 'Rot', 'Tran', 'Length Ratio', 'APE', 'RPE', 'Ave KP', 'Ave MP']
with pd.ExcelWriter('orb_result_all.xlsx', mode='a', if_sheet_exists='replace') as writer:
    df.to_excel(writer, sheet_name='trajectory_perturbation_rgbd')

# Filter by ratio of tracked lengths
# df1 = df[df['Length Ratio']>0.8].groupby('Method')['APE'].mean()
# df1 = df.groupby(['Rot', 'Tran'])['Length Ratio'].mean()
df1 = df.groupby(['Rot', 'Tran']).agg({'Length Ratio':['mean', 'std']})
with pd.ExcelWriter('orb_length_ratio.xlsx', mode='a', if_sheet_exists='replace') as writer1:
    df1.to_excel(writer1, sheet_name='trajectory_perturbation_rgbd')
print(df1)

             Length Ratio          
                     mean       std
Rot Tran                           
0   0            0.885372  0.066975
    0dot0125     0.489365  0.106986
    0dot025      0.246887  0.128577
    0dot05       0.128486  0.091127
1   0            0.661958  0.328845
    0dot0125     0.339963  0.189277
    0dot025      0.221513  0.156063
    0dot05       0.120412  0.068644
3   0            0.373553  0.329293
    0dot0125     0.096124  0.154055
    0dot025      0.087193  0.080761
    0dot05       0.059375  0.055568
5   0            0.413572  0.418438
    0dot0125     0.154607  0.159983
    0dot025      0.214462  0.111663
    0dot05       0.143885  0.039416
